## Clasificarea imaginilor folosind descriptori

In [381]:
import glob
import cv2 as cv
import numpy as np
from skimage.feature import hog
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from copy import deepcopy

### Incarcarea imaginilor de antrenare

In [382]:
path='train\\'
folders=glob.glob(path+'*')
print(folders)

['train\\soccer_ball', 'train\\yin_yang']


In [383]:
train_images=[]
train_labels=[]

In [384]:
for f in folders:
    clas=f.replace(path,'')
    print(clas)
    files=glob.glob(f+'\\*jpg')
    for i in files:
        image = cv.imread (i)
        image = cv.resize(image, (300, 300),interpolation = cv.INTER_LINEAR)
        img = np.asarray(image)
        train_images.append(img)
        train_labels.append(clas)
train_images=np.array(train_images)
train_labels=np.array(train_labels)

soccer_ball
yin_yang


In [385]:
print(train_images.shape)
print(train_labels.shape)

(94, 300, 300, 3)
(94,)


### Incarcarea imaginilor de testare

In [386]:
path='test\\'
folders=glob.glob(path+'*')
print(folders)

['test\\soccer_ball', 'test\\yin_yang']


In [387]:
test_images=[]
test_labels=[]

In [388]:
for f in folders:
    clas=f.replace(path,'')
    print(clas)
    files=glob.glob(f+'\\*jpg')
    for i in files:
        image = cv.imread (i)
        image = cv.resize(image, (300, 300),interpolation = cv.INTER_LINEAR)
        img = np.asarray(image)
        test_images.append(img)
        test_labels.append(clas)
test_images=np.array(test_images)
test_labels=np.array(test_labels)

soccer_ball
yin_yang


In [389]:
print(test_images.shape)
print(test_labels.shape)

(30, 300, 300, 3)
(30,)


### Diferite tipuri de descriptori

In [390]:
def get_flatten_images(images):
    images_flatten=[]
    for img in images:
        img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        img=img.flatten()
        images_flatten.append(img)
    images_flatten = np.array(images_flatten)
    return images_flatten

In [391]:
def get_gray_histogram(images):
    gray_histograms = []
    for img in images:
        img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        hist = cv.calcHist([img], [0], None, [256], [0, 256])
        gray_histograms.append(hist.flatten())
    gray_histograms = np.array(gray_histograms)
    return gray_histograms

In [392]:
def get_rgb_histogram(images):
    color_histograms = []
    for img in images:
        hist = cv.calcHist([img], [0, 1, 2],None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
        color_histograms.append(hist.flatten())
    color_histograms = np.array(color_histograms)
    return color_histograms

In [393]:
def get_hog_descriptors(images):
    descriptors = []
    for img in images:
        img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        features = hog(img, pixels_per_cell=(25, 25),orientations=9, cells_per_block=(2, 2), block_norm='L2-Hys', feature_vector=True)
        descriptors.append(features)
    descriptors = np.array(descriptors)
    return descriptors

### Calcularea descriptorilor pentru imaginile de antrenare si testare

In [394]:
train_images_flatten=get_flatten_images(train_images)
test_images_flatten=get_flatten_images(test_images)

In [395]:
train_images_gray_hist=get_gray_histogram(train_images)
test_images_gray_hist=get_gray_histogram(test_images)

In [396]:
train_images_color_hist=get_rgb_histogram(train_images)
test_images_color_hist=get_rgb_histogram(test_images)

In [397]:
train_images_hog=get_hog_descriptors(train_images)
test_images_hog=get_hog_descriptors(test_images)

### Antrenarea unui SVM liniar cu diferiti descriptori

In [398]:
best_accuracy = 0
best_c = 0
best_model2 = None
Cs = [10 ** -5, 10 ** -4,  10 ** -3,  10 ** -2, 10 ** -1, 10 ** 0]
for c in Cs:
    print('Antrenam un clasificator pentru c=%f' % c)
    model = LinearSVC(C=c)
    model.fit(train_images_hog, train_labels)
    acc = model.score(train_images_hog, train_labels)
    print(acc)
    if acc > best_accuracy:
        best_accuracy = acc
        best_c = c
        best_model2 = deepcopy(model)

print('Performanta clasificatorului optim pt c = %f' % best_c)

Antrenam un clasificator pentru c=0.000010
0.6808510638297872
Antrenam un clasificator pentru c=0.000100
0.7872340425531915
Antrenam un clasificator pentru c=0.001000
0.8829787234042553
Antrenam un clasificator pentru c=0.010000
0.9787234042553191
Antrenam un clasificator pentru c=0.100000
1.0
Antrenam un clasificator pentru c=1.000000
1.0
Performanta clasificatorului optim pt c = 0.100000


In [399]:
predicted_labels= best_model2.predict(test_images_hog)
print(accuracy_score(test_labels,predicted_labels))

0.9333333333333333
